In [1]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
import pandas as pd
import re
import numpy as np 
import math
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from deap import creator
from deap import base
from deap import tools
import tensorflow as tf

from deap import algorithms
import random

[nltk_data] Downloading package punkt to /home/spetz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2022-05-28 20:45:06.957797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-28 20:45:06.957820: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
########################TRAIN DATA#############################
path = '/home/spetz/Downloads/DeliciousMIL/Data/train-data.dat'


clean_files = []
df = pd.DataFrame()

file = open(path).readlines()
len(file)

#clear data
clear_file=[]
for i in range(len(file)):
    x=re.sub('<.*?>','',file[i])
    clear_file.append(x)

clear_file=clear_file[:1000]

#######X-TRAIN#######
#sequences word
clean_doc = []
wordfreq = {}
for doc in clear_file:
    tokens = nltk.word_tokenize(doc)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

import heapq
most_freq = heapq.nlargest(200, wordfreq, key=wordfreq.get)

In [3]:
word_idf_values = {}
for token in most_freq:
    doc_containing_word = 0
    for document in clear_file:
        if token in nltk.word_tokenize(document):
            doc_containing_word += 1
    word_idf_values[token] = np.log(len(clear_file)/(1 + doc_containing_word))


In [4]:
word_tf_values = {}
for token in most_freq:
    sent_tf_vector = []
    for document in clear_file:
        doc_freq = 0
        for word in nltk.word_tokenize(document):
            if token == word:
                  doc_freq += 1
        word_tf = doc_freq/len(nltk.word_tokenize(document))
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector

In [5]:
tfidf_values = []
for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append(tfidf_sentences)

In [6]:
tf_idf_model = np.asarray(tfidf_values)


In [7]:
X_train_tf_idf = np.transpose(tf_idf_model)
X_train_tf_idf.shape

(1000, 200)

In [8]:
########################TRAIN DATA#############################
path1 = '/home/spetz/Downloads/DeliciousMIL/Data/test-data.dat'


clean_files = []
df = pd.DataFrame()

file1 = open(path1).readlines()
len(file)

#clear data
clear_file1=[]
for i in range(len(file1)):
    x=re.sub('<.*?>','',file1[i])
    clear_file1.append(x)

clear_file1=clear_file1[:1000]

#######X-TRAIN#######
#sequences word
clean_doc = []
wordfreq = {}
for doc in clear_file:
    tokens = nltk.word_tokenize(doc)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

import heapq
most_freq1 = heapq.nlargest(200, wordfreq, key=wordfreq.get)


word_idf_values1 = {}
for token in most_freq1:
    doc_containing_word = 0
    for document in clear_file1:
        if token in nltk.word_tokenize(document):
            doc_containing_word += 1
    word_idf_values1[token] = np.log(len(clear_file1)/(1 + doc_containing_word))

word_tf_values1 = {}
for token in most_freq1:
    sent_tf_vector = []
    for document in clear_file1:
        doc_freq = 0
        for word in nltk.word_tokenize(document):
            if token == word:
                  doc_freq += 1
        word_tf = doc_freq/len(nltk.word_tokenize(document))
        sent_tf_vector.append(word_tf)
    word_tf_values1[token] = sent_tf_vector

tfidf_values1 = []
for token in word_tf_values1.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values1[token]:
        tf_idf_score = tf_sentence * word_idf_values1[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values1.append(tfidf_sentences)


In [9]:
tfidf_values1=np.asarray(tfidf_values1)
X_test_tf_idf=np.transpose(tfidf_values1)
X_test_tf_idf.shape

(1000, 200)

In [12]:

#load labels
labels_fnames = [
            '/home/spetz/Downloads/DeliciousMIL/Data/train-label.dat',
            '/home/spetz/Downloads/DeliciousMIL/Data/test-label.dat'
            ]

Y_train = pd.read_csv(labels_fnames[0] ,nrows=1000 ,delimiter = ' ', header = None)
Y_test= pd.read_csv(labels_fnames[1], nrows=1000,delimiter = ' ', header = None)


#len(test_labels) 3983
#len(train_labels) 8251


scaler = MinMaxScaler()
X_train_tf_idf = scaler.fit_transform(X_test_tf_idf)
X_test_tf_idf = scaler.fit_transform(X_test_tf_idf)

print(X_train_tf_idf.shape,X_test_tf_idf.shape,Y_train.shape,Y_test.shape)

(1000, 200) (1000, 200) (1000, 20) (1000, 20)


In [16]:
model=load_model('/home/spetz/Downloads/CE_model.h5/')

2022-05-28 20:51:17.532003: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-28 20:51:17.532027: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-28 20:51:17.532043: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (spetz-ZenBook-UX434IQ-Q407IQ): /proc/driver/nvidia/version does not exist
2022-05-28 20:51:17.532254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
# fitness function
def fitness(ind):
    loss, accuracy = model.evaluate(X_test_tf_idf, Y_test, verbose=0)
    # We want to penalty the individual depending on how many inputs he has.
    # Less inputs means more fit individual.
    # We also want the higher accuracy value, so the fitness value is a combination of both.
    # Higher fitness_ind will mean more fit individual, because we are based on accurcy.
    fitness_ind = accuracy - (np.count_nonzero(ind) / 1000) * 0.6
    return fitness_ind,

In [21]:
# Create and register functions ( code taken from the DEAP documentation)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
# Attribute generator
toolbox.register("attr_bool", random.randint, 0, 1)
# Structure initializers. 784 because every individual is a string of 784 bits.
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 1000)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
